In [1]:
import json
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from classifier import Classifier
from autoencoder import Autoencoder
from torchvision import datasets

In [10]:
results_files = os.listdir('./results')
class_results = [f for f in results_files if f.startswith('classifier') and f.endswith('.json')]
class_results.sort()

for file in class_results:
    with open(f"./results/{file}", 'r') as f:
        data = json.load(f)
        train_loss_inc = data['train_loss_incorrect']
        train_loss = data['train_loss']
        valid_loss = data['valid_loss']

        train_prec_inc = data['train_precision_incorrect']
        train_prec = data['train_precision']
        valid_prec = data['valid_precision']

    fig = plt.figure(figsize=(10,10))
    gs = fig.add_gridspec(2, hspace=0)
    ax = gs.subplots(sharex=True)

    ax[0].plot(train_loss_inc, label='Train Loss Incorrect')
    ax[0].plot(train_loss, label='Train Loss', linestyle='--')
    ax[0].plot(valid_loss, label='Validation Loss', linestyle='-.')
    ax[0].set_ylabel('Loss')
    ax[0].legend()
    ax[0].grid()

    ax[1].plot(train_prec_inc, label='Train Precision Incorrect')
    ax[1].plot(train_prec, label='Train Precision', linestyle='--')
    ax[1].plot(valid_prec, label='Validation Precision', linestyle='-.')
    ax[1].set_ylabel('Precision')
    ax[1].set_xlabel('Epoch')
    ax[1].legend()
    ax[1].grid()

    fig.tight_layout()
    plt.savefig(f'./results/{file[:-5]}.png')
    plt.close(fig)

In [3]:
# load the 3 classifier_i.pth models, and run the validation set through them
# then plot the confusion matrix for each of them

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([transforms.ToTensor()])

# Se cargan los datasets
valid_set_orig = datasets.FashionMNIST('MNIST_data/', download = True, train = False, transform = transform)

# Se cargan los dataloaders
valid_loader = DataLoader(valid_set_orig, batch_size = 100, shuffle = True, num_workers=os.cpu_count()-1)

# Se cargan los modelos
models = []
for i in range(3):
    base_model = Autoencoder(dropout=0.2, l_size=1024)
    model = Classifier(base_model, 10)
    model.load_state_dict(torch.load(f'./results/classifier_{i+1}.pth', map_location=device, weights_only=True))
    model.to(device)
    models.append(model)

In [11]:
# run the models on the validation set and plot the confusion matrix
for i, model in enumerate(models):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    cm = confusion_matrix(y_true, y_pred, normalize='true')
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=valid_set_orig.classes)
    fig, ax = plt.subplots(figsize=(12, 13))  # Increase the figure size
    disp.plot(ax=ax, colorbar=False)
    plt.savefig(f'./results/confusion_matrix_{i+1}.png')
    plt.close()